# Extraction et Création des fichiers txt contenant les articles
Clemence SEBE

In [13]:
import xml.etree.ElementTree as et 
import pandas as pd
import re
import os

# Extraction des Articles de PMC du fichier xml

In [14]:
pmcAll = et.parse('9_10_2022/q3_9_10_2022.xml')
pmc = pmcAll.getroot()

Fonction intermédiare permettant d'extraire certaines informations et le texte du XML : 

In [15]:
def extractText(part, sup=False):
    txt = ''
    if part.tag != 'fig' and part.tag != 'table' and part.tag != 'table-wrap':  
        
        if part.tag == 'list-item':
            for subPart in part:
                if subPart.tag != 'label':
                    txt += '** ' + extractText(subPart) + '\n'
        else:
            if sup:
                txt += ' ['
                
            if part.text != None:
                if len(part) == 0:
                    txt += part.text
                else: 
                    txt += (part.text).strip() + " "

            for subPart in part:
                if subPart.tag =='xref':
                    txt = txt.strip() 
                if subPart.tag == 'ext-link':
                    txt = txt.strip() + " "
                    
                if subPart.tag == 'sup':
                    txt = txt.strip()
                    txt += extractText(subPart, True)
                else:
                    txt += extractText(subPart)

            if sup:
                txt += ']'
            if part.tail != None:
                if len(part.tail.strip()) != 0:
                    txt += part.tail
    
    return txt

In [16]:
def extracTitle(body, tab, idx):
    section = body.findall('sec')
    for sec in section:
        title = sec.find('title')
        if title != None:
            txt = extractText(title)
               
            new = tab
            new.append([idx,txt])
            extracTitle(sec, new , idx+1)
        else:
            extracTitle(sec,tab, idx+1)
    return tab

In [17]:
def whichSection(titres, wordsAccepted):
    accepted = []
    for i in range (len(titres)) :
        word = titres[i][-1].lower()
        for wA in wordsAccepted:
            if word.find(wA) != -1:
                ok = True
                if titres[i][0] == 0:
                    if not titres[i][-1] in accepted:
                        accepted.append(titres[i][-1])
                else:
                    for j in range (i,-1,-1):
                        if titres[j][0] == 0:
                            if not titres[j][-1] in accepted:
                                accepted.append(titres[j][-1])
                            break
            
    return accepted

In [18]:
def extractTextSection(file, section):
    for part in section:
        if part.tag == 'fn-group':
            extractTextSection(file,part) 
        if part.tag == 'title':
            txt = '\n\n\t' + extractText(part) + '\n\n'
            file.write(txt)
        if part.tag == 'p' or part.tag == 'fn':
            txt = extractText(part) + '\n'
            file.write(txt)
        if part.tag == 'list':
            txt = extractText(part) 
            file.write(txt)
        if part.tag == 'notes':
            extractTextSection(file,part)
        if part.tag == 'sec':
            extractTextSection(file, part)

"Main" créant les différents fichier txt composant par la suite le corpus :

In [19]:
wordsAccepted = ['implementation', 'material', 'method', 'operation', 'pipeline', 'workflow', 'tool', '\u2003']
access = 0
nonAccess = 0
dicoArticle = {}
idx = 1

#a optimiser
try:
    os.mkdir("article")
    os.mkdir("article/all")
except:
    None


titre = []
titleSection = open("sectionTitle.txt", 'w')

paternGit = '(https:\/\/)?(www\.)?github\.com\/(\w|\/|-|_)*'

df = pd.DataFrame(columns=['id','titre', 'pmid', 'pmc', 'doi', 'XMLaccess', 'git', 'language'])                   

for article in pmc:
    print(str(idx) + "/" + str(len(pmc)))
    front = article.find('front')
    body = article.find('body')
    
    #extract gloabal information on the article
    dicoSubArticle = {}
    dicoSubArticle[idx] = {}
    articleMeta = front.find('article-meta')
    temp = {}
    for idA in articleMeta.findall('article-id'):
        if not idA.attrib['pub-id-type'] in temp:
            temp.update({idA.attrib['pub-id-type'] : idA.text})
        else:
            print('NORMALLY NO - NOT TWO ID DIFFERENT')
            
    for k in temp:
        dicoSubArticle[idx][k] = temp[k]
    
    title = articleMeta.find('title-group')
    txtTitle = []
    for child in title:
        if child.tag.find('title') != -1:
            txt = extractText(child)
            txtTitle.append(txt)
    dicoSubArticle[idx]['title'] = txtTitle
    
    #  abstract
    string = 'article/all/' + str(idx) + '.all.txt'
    fileAll = open(string,'w')
    fileAll.write(dicoSubArticle[idx]['title'][0] + '\n\n')
        
    abstract = articleMeta.findall('abstract')
    for i in range (len(abstract)):
        if i > 0:
            fileAll.write("\n" + "Abstract " + str(i) + ' :\n')
        else:
            fileAll.write("Abstract :\n")
        extractTextSection(fileAll, abstract[i])
    fileAll.write("\n" + "-"*100 + "\n")

    
    if body == None:
        nonAccess += 1
        dicoSubArticle[idx]['XMLaccess'] = 'No'
        dicoSubArticle[idx]['git'] = ''
        fileAll.close()
        stringGit = ''
        tabGit = []
    else:
        access += 1
        dicoSubArticle[idx]['XMLaccess'] = 'Free'
        
        #extract the text - only some sections       
        titres = extracTitle(body,[], 0)
        string = str(idx) + " "+ dicoSubArticle[idx]['title'][0]
        titleSection.write(string)
        titleSection.write('\n')
        for t in titres:
            titleSection.write('\t'*t[0])
            titleSection.write(t[1])
            titleSection.write('\n')
        titleSection.write('\n\n')
        
        nameSection = whichSection(titres, wordsAccepted)
        try:
            string = 'article/' + str(idx) + "_PMID" + dicoSubArticle[idx]['pmid'] + '.txt'
        except:
            string = 'article/' + str(idx) + "_PMC" + dicoSubArticle[idx]['pmc'] + '.txt'
        file = open(string,'w')
        file.write(dicoSubArticle[idx]['title'][0] + '\n')
        
        print(dicoSubArticle[idx]['title'][0])
        section = body.findall('sec')
        for sec in section:
            title = sec.find('title')
            if title != None:
                txt = extractText(title)
                if txt in nameSection:
                    extractTextSection(file,sec)
        file.close()
        
        #extract all the article
        for sec in section:
            extractTextSection(fileAll,sec)
        
        #and the section Data Availability if the article have one
        fileAll.write("\n" + "-"*100 + "\n")
        back = article.find('back')
        extractTextSection(fileAll, back)               
            
        fileAll.close() 
        
        #extract github @ 
        string = 'article/all/' + str(idx) + '.all.txt'
        fileR = open(string,'r')
        txt = fileR.read()
        tabGit = []
        stringGit = ''
        for match in re.finditer(paternGit, txt):
            git = txt[match.span()[0]: match.span()[1]]
            
            if git[0] == 'h':
                git = git[8:]
            if git[0] == 'w':
                git = git[4:]
            if not git in tabGit:
                tabGit.append(git)
                stringGit += git + ' , '
        stringGit = stringGit[:-(len(' , '))]
        dicoSubArticle[idx]['git'] = stringGit
        fileR.close()
        
            

    try:
        pmidid = dicoSubArticle[idx]['pmid'] 
    except :
        pmidid = ''
    try:
        pmcid = dicoSubArticle[idx]['pmc'] 
    except :
        pmcid = ''
    try:
        doiid = dicoSubArticle[idx]['doi'] 
    except :
        doiid = ''
        
    df = df.append({'id':idx, 'titre':dicoSubArticle[idx]['title'][0], 'pmid':pmidid, 'pmc':pmcid, 
                'doi':doiid, 'XMLaccess':dicoSubArticle[idx]['XMLaccess'], 'git':dicoSubArticle[idx]['git']}, ignore_index=True)
    
    print("~~~~~~~~~~~~~~~~~")
    
    #print(dicoSubArticle)
    dicoArticle.update(dicoSubArticle)
    idx += 1
    
#df.to_csv( language + "_corpus.csv", index=False)
#df.to_html(language + '_corpus.html',justify='center',index=False)
titleSection.close()

1/110
A bioinformatic-assisted workflow for genome-wide identification of ncRNAs
~~~~~~~~~~~~~~~~~
2/110
BioUML—towards a universal research platform
~~~~~~~~~~~~~~~~~
3/110
Detection and Validation of Circular DNA Fragments Using Nanopore Sequencing
~~~~~~~~~~~~~~~~~
4/110
GenErode: a bioinformatics pipeline to investigate genome erosion in endangered and extinct species
~~~~~~~~~~~~~~~~~
5/110
nf-gwas-pipeline: A Nextflow Genome-Wide Association Study Pipeline
~~~~~~~~~~~~~~~~~
6/110
~~~~~~~~~~~~~~~~~
7/110
NanoRTax, a real-time pipeline for taxonomic and diversity analysis of nanopore 16S rRNA amplicon sequencing data
~~~~~~~~~~~~~~~~~
8/110
MetaWorks: A flexible, scalable bioinformatic pipeline for high-throughput multi-marker biodiversity assessments
~~~~~~~~~~~~~~~~~
9/110
DetectIS: a pipeline to rapidly detect exogenous DNA integration sites using DNA or RNA paired-end sequencing data
~~~~~~~~~~~~~~~~~
10/110
TMBur: a distributable tumor mutation burden approach for whole genome

/tmp/ipykernel_6090/3205428600.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'id':idx, 'titre':dicoSubArticle[idx]['title'][0], 'pmid':pmidid, 'pmc':pmcid,
/tmp/ipykernel_6090/3205428600.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'id':idx, 'titre':dicoSubArticle[idx]['title'][0], 'pmid':pmidid, 'pmc':pmcid,
/tmp/ipykernel_6090/3205428600.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'id':idx, 'titre':dicoSubArticle[idx]['title'][0], 'pmid':pmidid, 'pmc':pmcid,
/tmp/ipykernel_6090/3205428600.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'id

~~~~~~~~~~~~~~~~~
62/110
Dadasnake, a Snakemake implementation of DADA2 to process amplicon sequencing data for microbial ecology
~~~~~~~~~~~~~~~~~
63/110
Accurate and Efficient KIR Gene and Haplotype Inference From Genome Sequencing Reads With Novel K-mer Signatures
~~~~~~~~~~~~~~~~~
64/110
Cascabel: A Scalable and Versatile Amplicon Sequence Data Analysis Pipeline Delivering Reproducible and Documented Results
~~~~~~~~~~~~~~~~~
65/110
Natrix: a Snakemake-based workflow for processing, clustering, and taxonomically assigning amplicon sequencing reads
~~~~~~~~~~~~~~~~~
66/110
~~~~~~~~~~~~~~~~~
67/110
Reproducibly sampling SARS-CoV-2 genomes across time, geography, and viral diversity
~~~~~~~~~~~~~~~~~
68/110
The Bacteria Genome Pipeline (BAGEP): an automated, scalable workflow for bacteria genomes with Snakemake
~~~~~~~~~~~~~~~~~
69/110
MetaLAFFA: a flexible, end-to-end, distributed computing-compatible metagenomic functional annotation pipeline
~~~~~~~~~~~~~~~~~
70/110
Hypercluster: a

/tmp/ipykernel_6090/3205428600.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'id':idx, 'titre':dicoSubArticle[idx]['title'][0], 'pmid':pmidid, 'pmc':pmcid,
/tmp/ipykernel_6090/3205428600.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'id':idx, 'titre':dicoSubArticle[idx]['title'][0], 'pmid':pmidid, 'pmc':pmcid,
/tmp/ipykernel_6090/3205428600.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'id':idx, 'titre':dicoSubArticle[idx]['title'][0], 'pmid':pmidid, 'pmc':pmcid,
/tmp/ipykernel_6090/3205428600.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'id

In [20]:
print("On {} articles, {} open access Articles and {} no XML open acess Articles".format(len(pmc),access, nonAccess))

On 110 articles, 99 open access Articles and 11 no XML open acess Articles


In [21]:
# A extraire : 
# implementation - Materials and Methods - Methods - Operation - Pipeline - Workflow - Library?? - Tool(s)
#Peut etre :
# Availability and requirements (*5)

# Deplacement des articles dans le bon dossier 

In [22]:
path = 'article/all/'
listArticle = os.listdir(path)

nbNextflow = 0
nbSnakemake = 0
nbBoth = 0
nonAccess = 0

"""try:
    os.mkdir('nextflow')
    os.mkdir('nextflow/all')
    os.mkdir('snakemake')
    os.mkdir('snakemake/all')
except:
    None"""
    

for article in listArticle:
    a = open(path + article, 'r')
    idxDf = int(article.split('.')[0]) -1
    txt = a.read()
    idx = txt.find('-'*100)
    abstract = txt[:idx].lower()

    n = abstract.find('nextflow')
    s = abstract.find('snakemake')
    
    if n != -1 and s == -1:
        df.loc[idxDf]['language'] = 'Nextflow'
        if df.loc[idxDf]['XMLaccess'] == 'Free':
            nbNextflow += 1
            """cmd = 'mv ' + path + article + " " + 'nextflow/all/'
            os.system(cmd)
            cmd = 'mv ' + ' article/' + str(idxDf+1) + "_*.txt" + " " + 'nextflow/'
            os.system(cmd)"""
        
    elif n == -1 and s != -1:
        df.loc[idxDf]['language'] = 'Snakemake'
        if df.loc[idxDf]['XMLaccess'] == 'Free':
            nbSnakemake += 1
            """cmd = 'mv ' + path + article + " " + 'snakemake/all/'
            os.system(cmd)
            cmd = 'mv ' + ' article/' + str(idxDf+1) + "_*.txt" +  " " + 'snakemake/'
            os.system(cmd)"""
            
            
    elif n != -1 and s != -1:
        nbBoth += 1
        df.loc[idxDf]['language'] = 'Both'
        
    a.close()
print("Nombre d'articles decrivant des wf Nextflow : {}".format(nbNextflow))
print("Nombre d'articles decrivant des wf Snakemake : {}".format(nbSnakemake))
print("Nombre d'articles possédant à la fois Nextflow et Snakemake : {}".format(nbBoth))


Nombre d'articles decrivant des wf Nextflow : 48
Nombre d'articles decrivant des wf Snakemake : 49
Nombre d'articles possédant à la fois Nextflow et Snakemake : 3


In [23]:
df.to_html('corpus.html',justify='center',index=False)

In [24]:
df

,id,titre,pmid,pmc,doi,XMLaccess,git,language
0,1,A bioinformatic-assisted workflow for genome-w...,35979446,9376865,10.1093/nargab/lqac059,Free,github.com/brummetheus/pinc,Nextflow
1,2,BioUML—towards a universal research platform,35536253,9252820,10.1093/nar/gkac286,Free,"github.com/Biosoft-ru/pybiouml , github.com/mo...",Nextflow
2,3,Detection and Validation of Circular DNA Fragm...,35711922,9195511,10.3389/fgene.2022.867018,Free,github.com/snakemake-workflows/cyrcular-calling,Snakemake
3,4,GenErode: a bioinformatics pipeline to investi...,35698034,9195343,10.1186/s12859-022-04757-0,Free,"github.com/NBISweden/GenErode , github.com/bro...",Snakemake
4,5,nf-gwas-pipeline: A Nextflow Genome-Wide Assoc...,35647481,9137404,10.21105/joss.02957,Free,github.com/montilab/nf-gwas-pipeline,Nextflow
...,...,...,...,...,...,...,...,...
105,106,ATAC2GRN: optimized ATAC-seq and DNase1-seq pi...,30064353,6069842,10.1186/s12864-018-4943-z,Free,"github.com/ChioriniLab , github.com/lh3/seqtk",Snakemake
106,107,Sequanix: a dynamic graphical interface for Sn...,29361152,5972652,10.1093/bioinformatics/bty034,Free,github.com/sequana/sequana,Snakemake
107,108,"TaxMapper: an analysis tool, reference databas...",29037173,5644092,10.1186/s12864-017-4168-6,Free,,Snakemake
108,109,MetaMeta: integrating metagenome analysis tool...,28807044,5557516,10.1186/s40168-017-0318-y,Free,"github.com/pirovc/metameta , github.com/biobox...",Snakemake
